In [0]:
import configparser
import glob
import os
import pandas as pd
import subprocess
import sys
import tarfile 
from urllib.request import urlretrieve
import json
import tempfile
import tensorflow as tf

In [25]:
!git clone  https://github.com/HisakaKoji/bert-japanese.git

Cloning into 'bert-japanese'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 299 (delta 10), reused 12 (delta 6), pack-reused 280
Receiving objects: 100% (299/299), 465.42 KiB | 5.82 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [26]:
%cd /content/bert-japanese/

/content/bert-japanese


In [27]:
!git clone https://github.com/google-research/bert.git

fatal: destination path 'bert' already exists and is not an empty directory.


In [28]:
%cd ..

/content


In [0]:
!pip install -q -r bert-japanese/requirements.txt

In [30]:
%cd bert-japanese/notebook

/content/bert-japanese/notebook


In [0]:
sys.path.append("../src")
from utils import str_to_value

In [0]:
sys.path.append("../bert")
import modeling

In [33]:
CURDIR = os.getcwd()
CONFIGPATH = os.path.join(CURDIR, os.pardir, 'config.ini')
config = configparser.ConfigParser()
config.read(CONFIGPATH)

['/content/bert-japanese/notebook/../config.ini']

In [0]:
from google.colab import auth
auth.authenticate_user()

In [35]:
!gsutil cp gs://hisaka/model/wiki-ja.model /content/bert-japanese/model/
!gsutil cp gs://hisaka/model/wiki-ja.vocab /content/bert-japanese/model/

Copying gs://hisaka/model/wiki-ja.model...
/ [1 files][786.8 KiB/786.8 KiB]                                                
Operation completed over 1 objects/786.8 KiB.                                    
Copying gs://hisaka/model/wiki-ja.vocab...
/ [1 files][581.7 KiB/581.7 KiB]                                                
Operation completed over 1 objects/581.7 KiB.                                    


In [36]:
!gsutil cp gs://hisaka/model/model.ckpt-1400000* /content/bert-japanese/model/

Copying gs://hisaka/model/model.ckpt-1400000.data-00000-of-00001...
Copying gs://hisaka/model/model.ckpt-1400000.index...
Copying gs://hisaka/model/model.ckpt-1400000.meta...
\ [3 files][  1.3 GiB/  1.3 GiB]   52.0 MiB/s                                   
Operation completed over 3 objects/1.3 GiB.                                      


In [0]:
FILEURL = config['FINETUNING-DATA']['FILEURL']
FILEPATH = config['FINETUNING-DATA']['FILEPATH']
EXTRACTDIR = config['FINETUNING-DATA']['TEXTDIR']

In [0]:
PRETRAINED_MODEL_PATH = '/content/bert-japanese/model/model.ckpt-1400000'

In [0]:
bert_config_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.json')
bert_config_file.write(json.dumps({k:str_to_value(v) for k,v in config['BERT-CONFIG'].items()}))
bert_config_file.seek(0)
bert_config_file_path = str(bert_config_file.name)
bert_config = modeling.BertConfig.from_json_file(bert_config_file.name)

In [0]:
!echo '政府への抗議活動が続く香港で１日午後、数千人規模のデモ隊が香港国際空港の周辺に集まり、ターミナルを包囲した。出入り口にバリケードを築くなどして空港利用者にも影響が出たが、約２時間後に警察が排除した。デモとの関係は不明だが、出発が数時間遅れた便もあり、空のダイヤに乱れが生じた。' > /tmp/input.txt

In [41]:
!git pull

Already up to date.


In [82]:
!python ../src/extract_features.py \
  --input_file=./test.txt \
  --output_file=./output.txt \
  --vocab_file=../model/wiki-ja.vocab \
  --model_file=../model/wiki-ja.model \
  --bert_config_file={bert_config_file.name} \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --layers=-1,-2,-3,-4 \
  --max_seq_length=128 \
  --batch_size=8



W0204 04:14:59.098333 140126425450368 module_wrapper.py:139] From ../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Loaded a trained SentencePiece model.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0204 04:14:59.266746 140126425450368 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

I0204 

In [48]:
!gsutil cp output.txt  gs://hisaka/

Copying file://output.txt [Content-Type=text/plain]...
/ [1 files][  7.9 KiB/  7.9 KiB]                                                
Operation completed over 1 objects/7.9 KiB.                                      


In [51]:
!ls

bert-japanese			   finetune-to-livedoor-corpus.ipynb
check-extract-features.ipynb	   output.txt
check-trained-tokenizer.ipynb	   pretraining.ipynb
finetune_to_livedoor_corpus.ipynb


In [52]:
!echo output.txt

output.txt


Let's have a look at the features of the last layer for the word "人間".

The 0-th token is always [CLS], and the 1st token of a sentence is [▁]. So the word comes in 3rd position.

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
import numpy as np
data1 = np.ndarray([])
i = 0
with open('./output.txt') as fin:
    for i,line in enumerate(fin):
        data = json.loads(line)
        value = data['features'][0]['layers'][0]['values']
        value = value/np.linalg.norm(value)
        value = np.round(value,2)
        if i == 0:
          data1 = value
        else:
          data1 = np.vstack((data1, value ) )
          
        if (i + 1 )% 1000 == 0:
          print(value[0])

In [84]:
data1

array([[ 0.04,  0.04,  0.06, ..., -0.01, -0.  ,  0.01],
       [ 0.  ,  0.04,  0.03, ..., -0.04,  0.01, -0.  ],
       [ 0.02,  0.08,  0.06, ..., -0.02, -0.02,  0.  ],
       ...,
       [-0.01,  0.04,  0.02, ..., -0.02, -0.  , -0.02],
       [ 0.02,  0.01, -0.02, ..., -0.  ,  0.03, -0.  ],
       [ 0.02,  0.01,  0.02, ..., -0.02,  0.02, -0.01]])

In [0]:
b = np.dot(data1,data1.T)

In [69]:
b

array([[1.0117, 0.6799, 0.8006, 0.6252, 0.6463, 0.58  , 0.6252, 0.6714,
        0.6405, 0.6016, 0.5482, 0.5813, 0.7019],
       [0.6799, 1.    , 0.7588, 0.6594, 0.6394, 0.7688, 0.716 , 0.5865,
        0.5763, 0.5847, 0.5373, 0.5404, 0.6324],
       [0.8006, 0.7588, 1.0151, 0.6685, 0.798 , 0.6438, 0.6308, 0.6083,
        0.6139, 0.5923, 0.5516, 0.5879, 0.7001],
       [0.6252, 0.6594, 0.6685, 1.0088, 0.6273, 0.6904, 0.6571, 0.5086,
        0.4656, 0.5272, 0.4878, 0.4639, 0.518 ],
       [0.6463, 0.6394, 0.798 , 0.6273, 0.9919, 0.6009, 0.5998, 0.5079,
        0.5123, 0.5603, 0.5342, 0.4993, 0.5777],
       [0.58  , 0.7688, 0.6438, 0.6904, 0.6009, 1.0046, 0.7537, 0.5236,
        0.4732, 0.4997, 0.4905, 0.4841, 0.543 ],
       [0.6252, 0.716 , 0.6308, 0.6571, 0.5998, 0.7537, 1.009 , 0.54  ,
        0.5036, 0.4965, 0.4842, 0.4477, 0.5283],
       [0.6714, 0.5865, 0.6083, 0.5086, 0.5079, 0.5236, 0.54  , 1.003 ,
        0.7126, 0.6714, 0.5834, 0.5616, 0.7093],
       [0.6405, 0.5763, 0.6139, 

In [0]:
b_sort = np.sort(b,axis=0)[::-1]

In [0]:
np.savetxt('_sort.txt',b_sort)

In [0]:
b_argsort = np.argsort(b,axis=0)[::-1]

In [0]:
np.savetxt('_argsort.txt',b_argsort )

The last year is layer 0, the one before is layer -1, etc...
The embeddings are stored in the *values* entry.

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('_argsort.txt',sep=' ',header = None,dtype=int)
dfsort = pd.read_csv('_sort.txt',sep=' ',header = None)
dfname = pd.read_csv('name.txt',sep=',',header = None )

In [0]:
name_dict = dfname.iloc[:,0].to_dict()

In [0]:
dfh = df.replace(name_dict)

In [0]:
dfsort = dfsort.drop(0)

In [0]:
df_concat = pd.concat([dfh, dfsort])

In [0]:
df_concat.to_csv('_match_result.csv' )

In [60]:
embeddings = outputs['features'][0]['layers'][0]['values']
print(embeddings)

[-0.233959, -0.325433, 0.089562, -0.386527, -0.47084, -0.520039, 0.48815, 0.440312, 0.197197, -0.074321, 1.099765, -0.387714, 0.253115, -0.236305, 0.174472, -0.235986, -0.003577, -0.262003, -0.407707, -0.232782, -0.516261, -0.233449, 0.32312, 0.823211, -0.35089, 0.006149, 0.777077, -0.484005, -0.196059, -0.631205, 0.536703, 0.272169, -0.093169, 0.652542, 0.604407, 0.617713, -0.328718, -0.239253, -3.769789, 0.456251, -0.442339, 0.594171, -0.116508, -0.045895, -0.034626, -0.184103, 0.040546, -0.057511, -0.761704, 0.690995, 0.20544, -0.36067, -0.176512, 0.376884, -0.100889, -0.363698, -0.125896, -0.305863, -0.149599, 0.238806, 0.594984, -0.024143, -0.080567, -0.076503, 0.05906, -0.21033, -0.260536, -0.071279, -0.568184, -0.746496, 0.742599, -0.431422, -0.195619, 0.507251, 0.09836, -0.434843, -0.204632, 0.555118, 0.231745, -0.443035, -0.377361, -0.633929, -0.155787, 0.005052, 0.158219, -0.108095, 0.407048, 0.497056, 0.414791, -0.364791, -0.126182, 0.233514, 0.195897, -0.183161, -0.148266, 

In [55]:
embeddings = outputs['features'][3]['layers'][0]['values']
print(len(embeddings))

IndexError: ignored